In [1]:
#!pip install transformers

# Импорт либ

In [2]:
import os
import math
import time
import random
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from functools import partial

from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
    get_linear_schedule_with_warmup,
    BertConfig
)

# Отключаем WandB и логи от Transformers
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_WANDB"] = "true"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Препроцессинг

In [4]:
file_path = "/content/drive/MyDrive/IMDB Dataset.csv"
df = pd.read_csv(file_path)
df['sentiment'] = df['sentiment'].map({'negative': 0, 'positive': 1})
df = df.head(10000) # использую маленькую выборку для более быстрого обучения

# Деление на train / val / test
train_val_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42)

print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

#Токенизатор берта
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class IMDBDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        review = self.data.iloc[idx]['review']
        label = self.data.iloc[idx]['sentiment']

        encoding = self.tokenizer(
            review,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        item = {key: tensor.squeeze(0) for key, tensor in encoding.items()}
        item['labels'] = torch.tensor(label, dtype=torch.long)
        return item

# Датасеты и даталоадеры
train_dataset = IMDBDataset(train_df, tokenizer)
val_dataset = IMDBDataset(val_df, tokenizer)
test_dataset = IMDBDataset(test_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

Train size: 5600
Validation size: 1400
Test size: 3000


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Alpha Nas

## Функция synthesize_candidate_from_mask
создает новую модель-кандидат на основе бинарной маски слоев. В контексте метода AlphaNAS эта функция выполняет этап синтеза по свойствам: из абстрактного описания архитектуры (список mask, где 1/0 означает оставить или убрать слой порождается конкретная архитектура. То есть, функция берет предобученный базовый BERT и удаляет из него те слои, которые не указаны в маске - реализуется шаг конкретизации архитектуры из набора свойств,(в статье говоритсяо б этом так: синтез подграфа по заданным свойствам, например "глубина сети = число оставленных слоев").

In [6]:
def synthesize_candidate_from_mask(mask: list, base_model: BertForSequenceClassification):
    """
    берем предобученную модель и оставляем только выбранные алгосом слои
    """
    candidate = copy.deepcopy(base_model)
    all_layers = candidate.bert.encoder.layer  # слои берт-бейз
    # берем только те слои , где mask[i] == 1
    pruned_layers = nn.ModuleList([layer for layer, m in zip(all_layers, mask) if m == 1])
    if len(pruned_layers) == 0:
        # если все слои отключены, оставим хотя бы первый (вылетала ошибка)
        pruned_layers = nn.ModuleList([all_layers[0]])
    candidate.bert.encoder.layer = pruned_layers
    candidate.config.num_hidden_layers = len(pruned_layers)
    print(f"Архитектура со слоями: {mask} , в ней {len(pruned_layers)} слоёв (из {len(all_layers)})")
    return candidate

## Функция mutate_mask
выполняет мутацию архитектуры, представленной маской. Это соответствует шагу мутации в эволюционном алгоритме AlphaNAS, где изменения вносятся на уровне абстрактных свойств. Здесь маска слоев служит "геномом" архитектуры. Функция случайным образом инвертирует отдельные биты маски с заданной вероятностью mutation_prob, тем самым включая или выключая соответствующие слои. После случайных изменений она проверяет и корректирует маску, чтобы количество активных слоев оставалось в допустимых пределах. Такие ограничения отражают требование к свойствам архитектуры: например, минимальная "глубина" 4 слоя.

In [7]:
def mutate_mask(mask: list, mutation_prob=0.3, min_layers=4, max_layers=12):
    """
    с заданной вероятностью меняет значение в каждом элементе маски (слои выбранные)
    После мутации проверяет, чтобы общее число включённых слоёв было не меньше минимально заданного
    и не больше максимально заданного числа
    """
    new_mask = mask.copy()
    for i in range(len(new_mask)):
        if random.random() < mutation_prob:
            new_mask[i] = 1 - new_mask[i]
    # Проверка ограничений по количеству слоёв
    num_layers = sum(new_mask)
    if num_layers < min_layers:
        indices = [i for i, val in enumerate(new_mask) if val == 0]
        random.shuffle(indices)
        for i in indices[:(min_layers - num_layers)]:
            new_mask[i] = 1
    elif num_layers > max_layers:
        indices = [i for i, val in enumerate(new_mask) if val == 1]
        random.shuffle(indices)
        for i in indices[:(num_layers - max_layers)]:
            new_mask[i] = 0
    return new_mask

## Функция count_parameters
вычисляет общее количество обучаемых параметров в модели
## Функция evaluate_candidate_by_mask
оценивает качество архитектуры-кандидата, заданной маской. Она реализует расчет фита модели для алгоритма поиска. Для AlphaNAS этот шаг соответствует оценке производительности сгенерированной архитектуры (после шага синтеза) по критериям точности и сложности.
## Алгоритм работы:
1. Синтезирует модель по данной маске (synthesize_candidate_from_mask) – получает конкретную архитектуру из абстрактного описания
2. Быстро обучает эту модель на обучающем наборе данных, чтобы частично адаптировать ее и оценить потенциал
3. Вычисляет accuracy на валидационном наборе и считает число параметров модели
4. На основе этих показателей рассчитывает кастомную метрику качества: accuracy минус штраф за сложность модели (λ * число параметров)
Таким образом, функция внедряет компромисс между точностью и размером модели

Также я наложил ограничения ограничения: если точность ниже заданного порога min_accuracy или число параметров превышает max_params, то архитектура сразу дисквалифицируется. Эти пороговые условия отражают требования, которые мы ставим для себя в рамках исследовния - избегать ухудшения качества ниже допустимого и соблюдать ограничение на ресурсы модели. По сути, эта функция выполняет роль оценки приспособленности кандидата в эволюционном процессе.

In [8]:
def count_parameters(model):
  #вычисляет общее количество обучаемых параметров в модели
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def evaluate_candidate_by_mask(mask: list, base_model: BertForSequenceClassification,
                               train_loader: DataLoader, val_loader: DataLoader,
                               epochs=1, device=device,
                               min_accuracy=0.75, max_params=1e8):
    #создаем кандидата по заданной маске: создаем копию базовой модели (задаем только количество слоев) с "обрезанными" слоями
    model = synthesize_candidate_from_mask(mask, base_model)
    model.to(device)
    # оптимизатор
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scaler = torch.cuda.amp.GradScaler()
    # обучаю модельку
    model.train()
    for epoch in range(epochs):
        for batch in tqdm(train_loader, desc=f"Training candidate (mask={mask})", leave=False):
            inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(**inputs, labels=labels)
                loss = outputs.loss
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        torch.cuda.empty_cache()
    # оцениваю модельку
    model.eval()
    total, correct = 0, 0
    for batch in tqdm(val_loader, desc=f"Evaluating candidate (mask={mask})", leave=False):
        inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)
        with torch.cuda.amp.autocast():
            outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(F.softmax(logits, dim=1), dim=1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
    accuracy = correct / total if total > 0 else 0.0
    # подсчитываю общее число обучаемых параметров модели
    params_count = count_parameters(model)

    # Проверяем ограничения
    if accuracy < min_accuracy:
        print(f"Отменяем архитектуру: аккураси {accuracy:.4f} < трешхолда {min_accuracy}")
        reward = -float('inf')
    elif params_count > max_params:
        print(f"Отменяем архитектуру: параметров модели {params_count} > трешхолда {max_params}")
        reward = -float('inf')
    else:
        lambda_coef = 1e-8
        reward = accuracy - lambda_coef * params_count

    print(f"Кандидат (mask = {mask}): аккураси: {accuracy:.4f}, параметров: {params_count}, кастомная метрика: {reward:.6f}")
    return reward, accuracy, params_count

## Функция evolutionary_search_by_mask
осуществляет эволюционный поиск лучшей архитектуры, представленной маской. Реализация основного цикла алгоритма AlphaNAS:
Инициализация: формируется начальная популяция масок (архитектур) заданного размера. Включается базовая маска (все слои включены, исходная модель) и несколько ее мутированных вариаций (случайные отключения слоев) для диверсификации. Эволюционный цикл на протяжении заданного числа поколений:
- Для каждой маски из текущей популяции вычисляется качество 
- Считаем кастомную метрику и показатели точности/размера для каждого кандидата
- Отбираются лучшие архитектуры поколения: результаты сортируются по reward, выводятся лучшие N
- Производится отбор и размножение: берутся топ-2 лучших маски и с помощью мутаций генерируются новые маски потомки, чтобы восстановить размер популяции. Лучшие архитектуры остаются и передают свои свойства потомкам.

По завершении всех поколений, хранится глобально лучшая найденная маска и ее показатели

In [13]:
def evolutionary_search_by_mask(base_model: BertForSequenceClassification,
                                train_loader: DataLoader, val_loader: DataLoader,
                                generations=3, population_size=4, device=device,
                                min_accuracy=0.75, max_params=1e8):
    """
    эволюционный поиск по бинарным маскам для включения слоев
    на каждой итерации оцениваются кандидаты, выбираются лучшие, и новые маски генерируются посредством мутации
    """
    num_total_layers = 12  # число начальных слоев
    base_mask = [1] * num_total_layers  # все слои включены
    population = [base_mask]
    for _ in range(population_size - 1):
        mutated = mutate_mask(base_mask, mutation_prob=0.5)
        population.append(mutated)

    best_mask = None
    best_reward = -float('inf')

    for gen in range(generations):
        print(f"\n Поколение {gen+1}")
        generation_results = []
        for mask in tqdm(population, desc="Оценка архитектур"):
            reward, acc, params = evaluate_candidate_by_mask(mask, base_model, train_loader, val_loader,
                                                               epochs=1, device=device,
                                                               min_accuracy=min_accuracy, max_params=max_params)
            generation_results.append((mask, reward, acc, params))
            if reward > best_reward:
                best_reward = reward
                best_mask = mask.copy()
        generation_results.sort(key=lambda x: x[1], reverse=True)
        print("Лучшие архитектуры поколения:")
        for i, (mask, rwd, acc, params) in enumerate(generation_results):
            print(f"{i+1}. Слои: {mask} -> Аккураси: {acc:.4f}, Параметров: {params}, Кастом: {rwd:.6f}")

        # отбираю топ-2 кандидата и генерируем новые маски посредством мутации
        new_population = []
        top_candidates = [mask for mask, _, _, _ in generation_results[:2]]
        new_population.extend(top_candidates)
        while len(new_population) < population_size:
            parent = random.choice(top_candidates)
            child = mutate_mask(parent, mutation_prob=0.3)
            new_population.append(child)
        population = new_population

    print("\n поиск завершен")
    print(f"Лучшая архитектура: {best_mask}")
    final_reward, final_acc, final_params = evaluate_candidate_by_mask(best_mask, base_model, val_loader, val_loader,
                                                                       epochs=1, device=device,
                                                                       min_accuracy=min_accuracy, max_params=max_params)
    return best_mask, final_reward, final_acc, final_params

In [15]:
def main():
    base_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
    base_model.to(device)

    # трешхолды
    required_min_accuracy = 0.75
    allowed_max_params = 1e8

    # поиск архитектуры
    best_mask, best_reward, best_accuracy, best_params = evolutionary_search_by_mask(
        base_model, train_loader, val_loader,
        generations=3, population_size=4, device=device,
        min_accuracy=required_min_accuracy, max_params=allowed_max_params
    )

    # Вывод итоговых результатов
    print("\n=== Итоговая архитектура ===")
    print(f"Слои: {best_mask}")
    print(f"аккураси: {best_accuracy:.4f}, параметры: {best_params}, кастомная метрика: {best_reward:.6f}")

In [16]:
main()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 Поколение 1


Оценка архитектур:   0%|          | 0/4 [00:00<?, ?it/s]

Архитектура со слоями: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] , в ней 12 слоёв (из 12)


<ipython-input-8-ebbe5c50c33e>:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


...

Evaluating candidate (mask=[0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]):   0%|          | 0/175 [00:00<?, ?it/s]

Кандидат (mask = [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]): аккураси: 0.8543, параметров: 59868674, кастомная метрика: 0.255599
Лучшие архитектуры поколения:
1. Слои: [0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1] -> Аккураси: 0.8536, Параметров: 52780802, Кастом: 0.325763
2. Слои: [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1] -> Аккураси: 0.8543, Параметров: 59868674, Кастом: 0.255599
3. Слои: [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1] -> Аккураси: 0.8436, Параметров: 59868674, Кастом: 0.244885
4. Слои: [0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1] -> Аккураси: 0.8614, Параметров: 66956546, Кастом: 0.191863

 поиск завершен
Лучшая архитектура: [0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
Архитектура со слоями: [0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1] , в ней 4 слоёв (из 12)


Training candidate (mask=[0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]):   0%|          | 0/175 [00:00<?, ?it/s]

Evaluating candidate (mask=[0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]):   0%|          | 0/175 [00:00<?, ?it/s]

Кандидат (mask = [0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]): аккураси: 0.7679, параметров: 52780802, кастомная метрика: 0.240049

=== Итоговая архитектура ===
Слои: [0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
аккураси: 0.7679, параметры: 52780802, кастомная метрика: 0.240049
